<a href="https://colab.research.google.com/github/lakshitgosain/Tensorflow-ZTM/blob/main/TF_ZTM_07_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Fundamentals in TF

Sequence-to-sequence
* One to one- Image Captioning
* Many to one- Sentiment analysis
* Many to one- time series forecasting
* many- to-many- Machine translation


In [1]:
!nvidia-smi -L

/bin/bash: nvidia-smi: command not found


In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-06-16 02:13:34--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-16 02:13:35 (54.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [4]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

--2023-06-16 02:13:40--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.218.128, 173.194.217.128, 173.194.215.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.218.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2023-06-16 02:13:40 (88.5 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

## Visualize the dataset

To visualize the sample data, we need to read the in. On way is to use python.

Another way to do this is use pandas



In [6]:
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

In [7]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
train_data_shuffled=train_df.sample(frac=1, random_state=42)

In [9]:
train_data_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [10]:
#Test dataframe looks like:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [11]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [12]:
#How many total number of samples
len(train_df), len(test_df)

(7613, 3263)

In [13]:
#Let's visualize some random training samples
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_data_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text, target=row
  print(f"Target: {target}", "(real disaster)" if target >0 else "(not real disaster)")
  print(f"text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
text:
burned 202 calories doing 30 minutes of Walking 4.0 mph very brisk pace #myfitnesspal

---

Target: 0 (not real disaster)
text:
Couples having less sex... for fear it'll be a let down: Internet movies and books saying how sex 'ought to be' pÛ_ http://t.co/c1xhIzPrAd

---

Target: 1 (real disaster)
text:
VICTORINOX SWISS ARMY DATE WOMEN'S RUBBER MOP WATCH 241487 http://t.co/yFy3nkkcoH http://t.co/KNEhVvOHVK

---

Target: 0 (not real disaster)
text:
What. The. Fuck. https://t.co/Nv7rK63Pgc

---

Target: 1 (real disaster)
text:
#LoMasVisto THOUSANDS OF HIPSTERS FEARED LOST: Giant Sinkhole Devours Brooklyn Intersectio... http://t.co/qwtk1b2fMC #CadenaDeSeguidores

---



### Split data into training and validation sets



In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_sentences, val_sentences, train_labels, val_labels=train_test_split(train_data_shuffled['text'].to_numpy(),
                                                                          train_data_shuffled['target'].to_numpy(),
                                                                          test_size=0.1,
                                                                          random_state=42)

In [16]:
#Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

## Converting words into numbers

First thing while building a model is to convert your text into numbers


Tokenization VS Embedding
Tkenization - word level and character level. Direct apping of a token.

Embedding - every word gets turned into a vector and we can define size of the vector. Embeddings can learn as our model trains



Text vectorization(tokenization)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [18]:
#Use the default text vectorization parameters
text_vectoizer= TextVectorization(max_tokens=None, #Defines how many words in our vocab
                               standardize= "lower_and_strip_punctuation",
                               split="whitespace",
                               ngrams=None,
                               output_mode='int',
                               output_sequence_length=None) #how long do you want your sequences to be
                               #pad_to_max_tokens=True)

In [19]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [20]:
#Find the average number of tokens(words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [21]:
max_vocab_length= 10000 #max no of words to have in our vocab
max_length=15 #Max length our sequences will be (e.g. how many words from a Tweet does a model see)

text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                                     output_mode="int",
                                     output_sequence_length=max_length)

In [22]:
#Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)


In [23]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

In [24]:
#Create a sample sentence and tokenize it
sample_sentence="There is a floow in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3,   1,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [25]:
random_sentence=random.choice(train_sentences)
print(random_sentence)
vectorized_random_sentence=text_vectorizer([random_sentence])
print(f"Random Sentence: {random_sentence}\nVectoized Random Sentence: {vectorized_random_sentence}")

.@StacDemon with five burning questions for Chris Mullin and St. JohnÛªs in 2015-16: http://t.co/NmRVTHkvAh #SJUBB
Random Sentence: .@StacDemon with five burning questions for Chris Mullin and St. JohnÛªs in 2015-16: http://t.co/NmRVTHkvAh #SJUBB
Vectoized Random Sentence: [[8152   14 1525   86 2012   10 5984    1    7  527    1    4    1    1
  8462]]


In [26]:
words_in_vocab=text_vectorizer.get_vocabulary()
top_5_words=words_in_vocab[:5]
botton_5_words=words_in_vocab[-5:]

In [27]:
words_in_vocab,top_5_words,botton_5_words

(['',
  '[UNK]',
  'the',
  'a',
  'in',
  'to',
  'of',
  'and',
  'i',
  'is',
  'for',
  'on',
  'you',
  'my',
  'with',
  'it',
  'that',
  'at',
  'by',
  'this',
  'from',
  'be',
  'are',
  'was',
  'have',
  'like',
  'as',
  'up',
  'so',
  'just',
  'but',
  'me',
  'im',
  'your',
  'not',
  'amp',
  'out',
  'its',
  'will',
  'an',
  'no',
  'has',
  'fire',
  'after',
  'all',
  'when',
  'we',
  'if',
  'now',
  'via',
  'new',
  'more',
  'get',
  'or',
  'about',
  'what',
  'he',
  'people',
  'news',
  'been',
  'over',
  'one',
  'how',
  'dont',
  'they',
  'who',
  'into',
  'were',
  'do',
  'us',
  '2',
  'can',
  'video',
  'emergency',
  'there',
  'disaster',
  'than',
  'police',
  'would',
  'his',
  'still',
  'her',
  'some',
  'body',
  'storm',
  'crash',
  'burning',
  'suicide',
  'back',
  'man',
  'california',
  'why',
  'time',
  'them',
  'had',
  'buildings',
  'rt',
  'first',
  'cant',
  'see',
  'got',
  'day',
  'off',
  'our',
  'going',
 

### Creating an embedding using an embedding layer

To make our embedding, we're going to use the TF's embedding layer.

The parameters we care the most about our embedding layers are:
* input_dim= sie of our vocab
* output_dim= size of the output embedding vector. A value of a 100 will mean that each token will be represented as a vector of 100 long length
* input_length= length of sequences being passed to the embedding layer
*

In [28]:
from tensorflow.keras import layers

In [29]:
embedding=layers.Embedding(input_dim=max_vocab_length,
                           output_dim=128,
                           input_length=max_length)

In [30]:
embedding

In [31]:
#Get a random sentence from the training set
random_sentence=random.choice(train_sentences)
print(f"Original Sentence:{random_sentence}\
      \nEmbedded Sentence")

sample_embedding=embedding(text_vectorizer([random_sentence]))
sample_embedding

Original Sentence:Our little forest fire wardens http://t.co/aPreNsss3x      
Embedded Sentence


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01826651, -0.0455185 ,  0.04478427, ..., -0.04277081,
         -0.02203444, -0.04961191],
        [ 0.04756333, -0.04445172,  0.01160629, ..., -0.00764678,
          0.04964563, -0.03272048],
        [-0.01116407, -0.03684238,  0.02412797, ...,  0.0487757 ,
          0.04302664, -0.02289302],
        ...,
        [-0.04026305, -0.01398071, -0.04515808, ..., -0.02679839,
         -0.04787008,  0.02381423],
        [-0.04026305, -0.01398071, -0.04515808, ..., -0.02679839,
         -0.04787008,  0.02381423],
        [-0.04026305, -0.01398071, -0.04515808, ..., -0.02679839,
         -0.04787008,  0.02381423]]], dtype=float32)>

In [32]:
#Check out a single token's embedding
sample_embedding[0][0], sample_embedding[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.01826651, -0.0455185 ,  0.04478427, -0.03212946, -0.01990367,
        -0.03452847,  0.02791091,  0.00127625, -0.04770733, -0.02936174,
        -0.01973971,  0.01447019, -0.02952641, -0.02820892, -0.01930742,
        -0.04286517,  0.04383672, -0.014073  ,  0.02084161, -0.03609059,
        -0.01494258,  0.03745918,  0.02149117, -0.02346244,  0.02442988,
         0.03049059,  0.03171021, -0.03821176,  0.02104301, -0.01027073,
         0.031429  , -0.04826967, -0.02511449, -0.01356739,  0.04092226,
         0.00390397, -0.00714183,  0.00134672,  0.03417296,  0.03022328,
        -0.01120727,  0.03472864, -0.00395777,  0.02994481,  0.04266192,
        -0.00809723, -0.00190679,  0.0080866 , -0.04014977, -0.03256259,
        -0.02171372,  0.03067607, -0.03380334, -0.02108996,  0.04747155,
         0.04911971,  0.00842319,  0.01081511,  0.03777042, -0.03020145,
         0.04646881, -0.04006764,  0.03422506, -0.02774705,  0.00046519,
  

##Modelling a text dataset(running a series of experiments)

* Model 0: Naive Bayes with Tokenization
* Model 1: Feed-forward Neural Network(Dense Model)
* Model 2: LSTM Model(RNN)
* Model 3: GRU Model
* Moel 4: Bidirectional LSTM
* Model 5 : 1D Convolutional Layer
* model 6: Tensorflow hub pretrained feature extraction (using transfer learning)
* Model 7 : Same as model 6 with 10% of data

How are we going to approach all of these:
Use the standard steps in modelling with tensorflow

* Create->build->fit->Evaluate



### Model 0

To create a baseline, we'll use sklearn's Naive Bayes using Tf-IDF to convert our words to numbers

It is a good practice to use non-DL Algos as a baseline because of their speed and then later use DL to see if you can improve upon them

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [34]:
model_0=Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [35]:
#Evaluate the baseline Model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"our baseline model achieves an accuracy of : {baseline_score*100:.2f}%")

our baseline model achieves an accuracy of : 79.27%


In [36]:
train_df.target.value_counts

<bound method IndexOpsMixin.value_counts of 0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64>

In [37]:
#Make predictions
baseline_preds= model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Create an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time.
Calculate:
* Accuracy
* Precision
* Recall
* F1 score


In [38]:
#Function to evaluate the above metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculated model accuracy, precision, recall and f1 score of a binary classification model
  """
  #Calculate model accuracy
  model_accuracy=accuracy_score(y_true, y_pred) * 100
  #Calculate model precision, recall and f1 score using the"weighted average"
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred,average="weighted" )
  model_results={'accuracy':model_accuracy,
                 'precision': model_precision,
                 'recall': model_recall,
                 'f1-score':model_f1}
  return model_results

In [39]:
#Get baseline results
baseline_results=calculate_results(y_true=val_labels,
                                   y_pred=baseline_preds)

In [40]:
baseline_results['accuracy']

79.26509186351706

In [41]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1-score': 0.7862189758049549}

## Model 1: A simple Dense Model

In [42]:
#Create a tensorflow callback
from helper_functions import create_tensorboard_callback

#Create a directory to save tensorboard logs
SAVE_DIR="model_logs"


In [43]:
#Build model with functional API
from tensorflow.keras import layers

inputs=layers.Input(shape=(1,), dtype=tf.string) #Inputs are 1 dimensional
x= text_vectorizer(inputs) #Turn the input text into numbers
x= embedding(x) #Create an embedding of vctorized embeddings
x=layers.GlobalAveragePooling1D()(x)
outputs=layers.Dense(1, activation='sigmoid')(x)

model_1=tf.keras.Model(inputs, outputs, name="Model_1_dense")

In [44]:
model_1.summary()

Model: "Model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [45]:
#Compile model
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [46]:
model_1_history=model_1.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                   experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230616-021343
Epoch 1/5
215/215 [==============================] - 9s 36ms/step - loss: 0.6087 - accuracy: 0.6970 - val_loss: 0.5365 - val_accuracy: 0.7480
Epoch 2/5
215/215 [==============================] - 7s 33ms/step - loss: 0.4413 - accuracy: 0.8212 - val_loss: 0.4733 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 8s 38ms/step - loss: 0.3470 - accuracy: 0.8594 - val_loss: 0.4605 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 7s 33ms/step - loss: 0.2841 - accuracy: 0.8905 - val_loss: 0.4612 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 8s 36ms/step - loss: 0.2370 - accuracy: 0.9115 - val_loss: 0.4769 - val_accuracy: 0.7900


In [47]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4769 - accuracy: 0.7900


[0.47688421607017517, 0.7900262475013733]

In [48]:
model_1_pred_probs=model_1.predict(val_sentences)

24/24 [==============================] - 0s 2ms/step


In [49]:
model_1_pred_probs.shape

(762, 1)

In [50]:
calculate_results(val_labels, model_1_pred_probs)

ValueError: ignored

In [51]:
val_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [52]:
#Convert Model_prediction Probabs into label formats
model_1_preds=tf.squeeze(tf.round(model_1_pred_probs))

In [53]:
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [54]:
model_1_results=calculate_results(y_true=val_labels,
                                  y_pred=model_1_preds)

In [55]:
model_1_results

{'accuracy': 79.00262467191601,
 'precision': 0.795059849830577,
 'recall': 0.7900262467191601,
 'f1-score': 0.7870730838048355}

In [56]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1-score': 0.7862189758049549}

In [57]:
import numpy as np
np.array(list(model_1_results.values()))> np.array(list(baseline_results.values()))

array([False, False, False,  True])

## Visualize the learned embeddings


In [ ]:
#Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()

In [ ]:
len(words_in_vocab), words_in_vocab[:10]

In [ ]:
# Model 1 summary
model_1.summary()

In [ ]:
#Get the weight matrix of the embedding layers
#These are numerical representation of our numerical datawhich are trained for 5 epochs

embed_weights=model_1.get_layer("embedding").get_weights()[0]
embed_weights

In [ ]:
print(embed_weights.shape)

Now we've got the embedding matrix our model has learned to represent tokens, let's see how we can visualize it

To do so, tensorflow has a tool called Projector

In [ ]:
# # Code below is adapted from: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
import io

# Create output writers
out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# Write embedding vectors and words to file
for num, word in enumerate(words_in_vocab):
  if num == 0:
     continue # skip padding token
  vec = embed_weights[num]
  out_m.write(word + "\n") # write words to file
  out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
out_v.close()
out_m.close()

In [ ]:
# Download files locally to upload to Embedding Projector
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download("embedding_vectors.tsv")
  files.download("embedding_metadata.tsv")

## Recurrent Neural Networks(RNNs)

the premise of the Recurrent Neural Networs is to use the representation of the previous input to aid the representation of the later input

RNNs are used for sequence data (this example is a sequence of text)

 📖 Resources:

MIT Deep Learning Lecture on Recurrent Neural Networks - explains the background of recurrent neural networks and introduces LSTMs.
The Unreasonable Effectiveness of Recurrent Neural Networks by Andrej Karpathy - demonstrates the power of RNN's with examples generating various sequences.
Understanding LSTMs by Chris Olah - an in-depth (and technical) look at the mechanics of the LSTM cell, possibly the most popular RNN building block.

### Model 2: LSTM

LSTM= long short term memory

Our structure of an RNN Looks like this

Input(text)-> Tokenize -> Embedding -> Layers(RNNs/Dense)->output(label probability)

In [63]:
#Create an LSTM Model
from tensorflow.keras import layers
inputs= layers.Input(shape=(1,), dtype=tf.string)
x=text_vectorizer(inputs)
x= embedding(x)
print(x.shape)
x=layers.LSTM(64, return_sequences=True)(x) #64 is hidden units
#When you a stacking RNN Cells together, you need to return sequences=true
print(x.shape)
x= layers.LSTM(64)(x)
print(x.shape)
x= layers.Dense(64, activation='relu')(x)
print(x.shape)
outputs=layers.Dense(1, activation='sigmoid')(x)

model_2= tf.keras.Model(inputs, outputs, name="model_2_LSTM")


(None, 15, 128)
(None, 15, 64)
(None, 64)
(None, 64)


In [64]:
#Create an LSTM Model
from tensorflow.keras import layers
inputs= layers.Input(shape=(1,), dtype=tf.string)
x=text_vectorizer(inputs)
x= embedding(x)
#print(x.shape)
#x=layers.LSTM(64, return_sequences=True)(x) #64 is hidden units
#When you a stacking RNN Cells together, you need to return sequences=true
#print(x.shape)
x= layers.LSTM(64)(x)
#print(x.shape)
#x= layers.Dense(64, activation='relu')(x)
#print(x.shape)
outputs=layers.Dense(1, activation='sigmoid')(x)

model_2= tf.keras.Model(inputs, outputs, name="model_2_LSTM")


In [65]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       multiple                  1280000   
                                                                 
 lstm_12 (LSTM)              (None, 64)                49408     
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

###The default activation function for RNNs is TanH

In [68]:
#Complie the model
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [69]:
history_model_2_LSTM=model_2.fit(train_sentences,
                                 train_labels,
                                 epochs=5,
                                 validation_data=(val_sentences, val_labels),
                                 callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                        experiment_name="model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230616-023831
Epoch 1/5
215/215 [==============================] - 12s 43ms/step - loss: 0.2179 - accuracy: 0.9232 - val_loss: 0.5918 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 8s 36ms/step - loss: 0.1578 - accuracy: 0.9400 - val_loss: 0.5878 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 11s 49ms/step - loss: 0.1283 - accuracy: 0.9508 - val_loss: 0.8088 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 12s 56ms/step - loss: 0.1063 - accuracy: 0.9594 - val_loss: 0.8088 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 12s 55ms/step - loss: 0.0887 - accuracy: 0.9653 - val_loss: 0.9141 - val_accuracy: 0.7782


In [70]:
model_2_pred_probs=model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 2s 8ms/step


array([[9.8220157e-03],
       [4.7423866e-01],
       [9.9969208e-01],
       [4.2027272e-02],
       [3.5633449e-04],
       [9.9869198e-01],
       [6.2313449e-01],
       [9.9983722e-01],
       [9.9966562e-01],
       [5.9080404e-01]], dtype=float32)

In [71]:
#Convert the probabs to pred
model_2_preds=tf.squeeze(tf.round(model_2_pred_probs))

In [72]:
model_2_preds


<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [73]:
model_2_results=calculate_results(y_true=val_labels,
                                  y_pred=model_2_preds)

In [74]:
model_2_results

{'accuracy': 77.82152230971128,
 'precision': 0.7804499491198106,
 'recall': 0.7782152230971129,
 'f1-score': 0.7760126933653841}

In [75]:
baseline_results


{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1-score': 0.7862189758049549}

### Model 3: GRU

Similar to LSTM but has less params

In [81]:
from tensorflow.keras import layers
inputs= tf.keras.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)

x=embedding(x)

x = tf.keras.layers.GRU(64, return_sequences=True)(x) #if you want to stack recurrent layers on top of each other, you need to use return_sequences=True
x=layers.LSTM(64, return_sequences=True)(x)
x=layers.GRU(64)(x)

x=layers.Dense(64, activation='relu')(x)
outputs=layers.Dense(1, activation='sigmoid')(x)
model_3=tf.keras.Model(inputs, outputs, name="Model_3_GRU")


In [85]:
from tensorflow.keras import layers
inputs= tf.keras.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)

x=embedding(x)

x = tf.keras.layers.GRU(64)(x) #if you want to stack recurrent layers on top of each other, you need to use return_sequences=True
#x=layers.LSTM(64, return_sequences=True)(x)
#x=layers.GRU(64)(x)

x=layers.Dense(64, activation='relu')(x)
outputs=layers.Dense(1, activation='sigmoid')(x)
model_3=tf.keras.Model(inputs, outputs, name="Model_3_GRU")


In [86]:
model_3.summary()

Model: "Model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       multiple                  1280000   
                                                                 
 gru_7 (GRU)                 (None, 64)                37248     
                                                                 
 dense_14 (Dense)            (None, 64)                4160      
                                                                 
 dense_15 (Dense)            (None, 1)                 65        
                                                       

In [87]:
model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
history_model_3_GRU=model_3.fit(train_sentences,
                                train_labels,
                                validation_data=(val_sentences, val_labels),
                                epochs=5,
                                callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                      experiment_name='model_name_GRU')])

Saving TensorBoard log files to: model_logs/model_name_GRU/20230616-025921
Epoch 1/5
